# 📘 Dynamic Auto Loader Project
This notebook ingests files like `edm_entity1.txt`, `edm_sub1.txt` from S3 and dynamically routes them to Delta tables based on filename. Each table will be auto-created inside its respective schema if not present.

In [ ]:
# 📦 Step 1: Setup
from pyspark.sql.functions import input_file_name, regexp_extract, col
import re

# 📍 Organization and S3 config
org_name = "entity_resolution_dev"
bucket = "your-s3-bucket"
s3_base_input = f"s3://{bucket}/autoloader-input/{org_name}/"
s3_base_output = f"s3://{bucket}/delta/{org_name}/"

# 📍 Watch all files in org folder
df = (spark.readStream
      .format("cloudFiles")
      .option("cloudFiles.format", "text")
      .load(s3_base_input)
     )

In [ ]:
# 🧠 Step 2: Parse Schema and Table Name From File Name
df = df.withColumn("source_file", input_file_name())
df = df.withColumn("file_base", regexp_extract("source_file", r"([^/]+)\.txt$", 1))
df = df.withColumn("schema_name", regexp_extract("file_base", r"([a-zA-Z_]+)[0-9a-z]*", 1))
df = df.withColumn("database", col("schema_name"))
df = df.withColumn("table", col("schema_name"))

df.select("source_file", "file_base", "database", "table").distinct().show()

In [ ]:
# ⚙️ Step 3: Auto Create Schema/Table Function
def ensure_schema_table(df_sample, database, table, output_path):
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {database}")
    full_table_name = f"{database}.{table}"

    if not spark.catalog.tableExists(full_table_name):
        print(f"🆕 Creating table: {full_table_name}")
        df_sample.write.format("delta").mode("overwrite").save(output_path)
        spark.sql(f"""
            CREATE TABLE {full_table_name}
            USING DELTA
            LOCATION '{output_path}'
        """)
    else:
        print(f"✅ Table {full_table_name} already exists.")

In [ ]:
# ⚡ Step 4: Split by Schema/Table and Write Stream to Correct Destination
schema_table_pairs = df.select("database", "table").distinct().collect()

for row in schema_table_pairs:
    database = row["database"]
    table = row["table"]
    output_path = f"{s3_base_output}{database}/{table}/"
    filtered_df = df.filter((col("database") == database) & (col("table") == table))

    parsed_df = filtered_df.selectExpr("split(value, ',') as columns")\
                           .selectExpr("columns[0] as col1", "columns[1] as col2")

    ensure_schema_table(parsed_df.limit(1), database, table, output_path)

    (parsed_df.writeStream
        .format("delta")
        .outputMode("append")
        .option("checkpointLocation", f"{output_path}/_checkpoint")
        .start(output_path))